## Phần 2: Tiền xử lý Dữ liệu (Data Preprocessing)

Tiền xử lý là giai đoạn thiết yếu để chuyển đổi dữ liệu thô thành định dạng phù hợp, đảm bảo độ tin cậy trước khi đưa vào huấn luyện mô hình. Quy trình này được thực hiện tuần tự qua các bước sau:

### 1. Tách biến Mục tiêu và Đặc trưng
Để thiết lập bài toán dự đoán, dữ liệu cần được phân tách rõ ràng. Cột `Attrition_Flag` (biến mục tiêu - target) sẽ được tách riêng ra khỏi các biến còn lại, đảm bảo tính khách quan trong quá trình huấn luyện.

### 2. Xử lý Dữ liệu thiếu (Missing Values)
Từ kết quả phân tích khám phá (EDA), chúng ta ghi nhận sự xuất hiện của giá trị `'Unknown'` trong hai cột `Education_Level` và `Income_Category`. Để duy trì kích thước mẫu dữ liệu mà không làm sai lệch phân phối, giải pháp tối ưu là thay thế các giá trị này bằng **mode** (giá trị xuất hiện tần suất cao nhất) của từng cột tương ứng.

### 3. Mã hóa biến phân loại (Categorical Encoding)
Do các thuật toán học máy yêu cầu đầu vào là dữ liệu số, các biến định danh dạng chữ cần được chuyển đổi thông qua hai kỹ thuật chính:

* **Label Encoding:** Áp dụng cho các biến nhị phân (như `Gender`). Các giá trị văn bản sẽ được ánh xạ trực tiếp sang 0 và 1.
* **One-Hot Encoding:** Áp dụng cho các biến danh mục có nhiều hơn hai giá trị (như `Marital_Status`). Phương pháp này tạo ra các cột giả (dummy variables) cho từng trạng thái, giúp mô hình tránh hiểu nhầm về thứ tự ưu tiên giữa các danh mục.

### 4. Đồng bộ thang đo (Feature Scaling)
Dữ liệu hiện tại có sự chênh lệch lớn về độ lớn giá trị (ví dụ: `Credit_Limit` lên tới hàng chục nghìn, trong khi `Dependent_count` chỉ là số đơn vị nhỏ). Điều này dễ khiến mô hình bị thiên kiến về các biến có giá trị lớn.

Để khắc phục, phương pháp **Standardization (Chuẩn hóa Z-score)** được áp dụng để đưa tất cả các biến số về cùng một phân phối chuẩn, giúp mô hình hội tụ nhanh và chính xác hơn.

### 5. Tổng hợp và Phân chia tập dữ liệu (Train-Test Split)
Sau khi xử lý xong các đặc trưng số và phân loại, dữ liệu được gộp lại thành một ma trận đặc trưng ($X$) hoàn chỉnh. Bước cuối cùng là phân chia tập dữ liệu để kiểm chứng độ hiệu quả:

* **Tập Huấn luyện (Training set - 80%):** Dùng để xây dựng và tối ưu hóa tham số mô hình.
* **Tập Kiểm tra (Testing set - 20%):** Dùng làm thước đo khách quan để đánh giá khả năng dự báo của mô hình trên dữ liệu thực tế chưa từng tiếp xúc.

In [2]:
# Cell 1: Import và thiết lập
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
sys.path.append('../')

# Import các hàm từ file exploration và preprocessing
from src.data_exploring import load_dataset
from src.data_processing import (
    clean_and_separate_target,
    handle_missing_values,
    encode_categorical_features,
    scale_numerical_features,
    train_test_split,
    save_processed_data
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Cell 2: Tải và Phân loại Cột (giống như trước)
# Tải dữ liệu
filepath = '../data/raw/BankChurners.csv'
raw_data = load_dataset(filepath)
raw_data = raw_data[list(raw_data.dtype.names)[:-2]] # Bỏ 2 cột cuối

# Phân loại cột
target_col = 'Attrition_Flag'
cols_to_exclude = ['CLIENTNUM']
all_cols = raw_data.dtype.names
numerical_cols = [name for name in all_cols if np.issubdtype(raw_data.dtype[name], np.number) and name not in cols_to_exclude]
categorical_cols = [name for name in all_cols if not np.issubdtype(raw_data.dtype[name], np.number) and name not in [target_col] + cols_to_exclude]


Tải dữ liệu thành công từ ../data/raw/BankChurners.csv


In [4]:
# Cell 3: Thực hiện Pipeline Tiền xử lý
print("BẮT ĐẦU QUÁ TRÌNH TIỀN XỬ LÝ\n" + "="*40)

# 1. Tách biến mục tiêu y và các đặc trưng features
features, y = clean_and_separate_target(raw_data, target_col)
print(f"\n1. Đã tách biến mục tiêu 'y' (shape: {y.shape})")

# 2. Xử lý dữ liệu thiếu trong các cột categorical
cols_with_unknown = ['Education_Level', 'Income_Category']
features = handle_missing_values(features, cols_with_unknown)
print("\n2. Đã xử lý giá trị thiếu.")

# 3. Mã hóa các cột categorical
encoded_categorical_data = encode_categorical_features(features, categorical_cols)
print(f"\n3. Đã mã hóa các cột categorical (shape mới: {encoded_categorical_data.shape})")

# 4. Chuẩn hóa các cột numerical
scaled_numerical_data = scale_numerical_features(features, numerical_cols)
print(f"4. Đã chuẩn hóa các cột numerical (shape mới: {scaled_numerical_data.shape})")

# 5. Kết hợp tất cả các đặc trưng đã xử lý thành ma trận X
X = np.hstack((scaled_numerical_data, encoded_categorical_data))
print(f"\n5. Đã kết hợp thành ma trận cuối cùng 'X' (shape: {X.shape})")

print("\nQUÁ TRÌNH TIỀN XỬ LÝ HOÀN TẤT!\n" + "="*40)


BẮT ĐẦU QUÁ TRÌNH TIỀN XỬ LÝ

1. Đã tách biến mục tiêu 'y' (shape: (10127,))
 - Cột 'Education_Level': Thay thế 'Unknown' bằng mode là 'Graduate'.
 - Cột 'Income_Category': Thay thế 'Unknown' bằng mode là 'Less than $40K'.

2. Đã xử lý giá trị thiếu.
 - Cột 'Gender': Mã hóa Label (0/1).
 - Cột 'Education_Level': Mã hóa One-Hot với 5 cột mới.
 - Cột 'Marital_Status': Mã hóa One-Hot với 3 cột mới.
 - Cột 'Income_Category': Mã hóa One-Hot với 4 cột mới.
 - Cột 'Card_Category': Mã hóa One-Hot với 3 cột mới.

3. Đã mã hóa các cột categorical (shape mới: (10127, 16))
4. Đã chuẩn hóa các cột numerical (shape mới: (10127, 14))

5. Đã kết hợp thành ma trận cuối cùng 'X' (shape: (10127, 30))

QUÁ TRÌNH TIỀN XỬ LÝ HOÀN TẤT!


In [5]:
# Cell 4: Phân chia dữ liệu thành tập Train và Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Đã phân chia dữ liệu:")
print(f" - X_train shape: {X_train.shape}")
print(f" - y_train shape: {y_train.shape}")
print(f" - X_test shape: {X_test.shape}")
print(f" - y_test shape: {y_test.shape}")

Đã phân chia dữ liệu:
 - X_train shape: (8102, 30)
 - y_train shape: (8102,)
 - X_test shape: (2025, 30)
 - y_test shape: (2025,)


In [6]:
#  Định nghĩa đường dẫn lưu file
# Lưu ý: '../' để thoát khỏi folder notebooks, đi vào data/processed
save_path = '../data/processed/bank_churn_processed.npz'

#  Gọi hàm lưu
# (Đảm bảo bạn đã có các biến X_train, X_test, y_train, y_test từ các bước trên)
save_processed_data(X_train, X_test, y_train, y_test, save_path)

Đã lưu dữ liệu thành công vào: ../data/processed/bank_churn_processed.npz
